# General

This notebook is used to calculate and plot ensemble temperature figures for PMIP4 midHolocene paper.

### Import packages and define fucntions for calculations

In [1]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF


In [2]:
#Define PMIP generations and model names
pmip_v4='PMIP4'
pmip_v3='PMIP3'
pmip={}
pmip['PMIP4']=['AWI-ESM-1-1-LR',
               'CESM2',
               'EC-Earth3-LR',
               'FGOALS-f3-L',
               'FGOALS-g3',
               'GISS-E2-1-G',
               'HadGEM3-GC31',
               'INM-CM4-8',
               'IPSL-CM6A-LR',
               'MIROC-ES2L',
               'MPI-ESM1-2-LR',
               'MRI-ESM2-0',
               'NESM3',
               'NorESM1-F',
               'NorESM2-LM',
               'UofT-CCSM-4']
pmip['PMIP3']=['BCC-CSM1-1',
               'CCSM4',
               'CNRM-CM5',
               'CSIRO-Mk3L-1-2',
               'CSIRO-Mk3-6-0',
               'EC-EARTH-2-2',
               'FGOALS-g2',
               'FGOALS-s2',
               'GISS-E2-R',
               'HadGEM2-CC',
               'HadGEM2-ES',
               'IPSL-CM5A-LR',
               'MIROC-ESM',
               'MPI-ESM-P',
               'MRI-CGCM3']
               
               
  




In [3]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict
    


# Create files

## 180x360 Geo2D data files (individual models and ave and std)

In [4]:
def ensemble_mean(pmip_v,variable_name,experiment_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    gcm_dict=ensemble_members_dict(variable_name,experiment_name)

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in gcm_dict:
        if gcm in pmip[pmip_v]:
            n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
    n=0
    for gcm in gcm_dict:
        if gcm in pmip[pmip_v]:
            modellist.append(gcm)
            this_file=xr.open_dataset(gcm_dict.get(gcm),decode_times=False)
            this_var=this_file[variable_name]
            this_regridder=xe.Regridder(this_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
            var_1x1=this_regridder(this_var)
            this_file.close()
            data[n,:,:]=var_1x1
            n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      
    return modellist

In [5]:
filename='../outputs/netcdf/PMIP4_midHolocene_monsoon.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='monsoon_summer_rainrate'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc


/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM']

In [6]:
filename='../outputs/netcdf/PMIP3_midHolocene_monsoon.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='monsoon_summer_rainrate'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Create weight file: bilinear_128x256_180x360_peri.nc


/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_96x192_180x360_peri.nc
Create weight file: bilinear_56x64_180x360_peri.nc


/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_160x320_180x360_peri.nc
Create weight file: bilinear_60x128_180x360_peri.nc


...

/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_90x144_180x360_peri.nc
Create weight file: bilinear_145x192_180x360_peri.nc


/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


Reuse existing file: bilinear_145x192_180x360_peri.nc
Create weight file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


/srv/conda/envs/notebook/lib/python3.7/site-packages/xesmf/frontend.py:421: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


In [7]:
filename='../outputs/netcdf/PMIP4_piControl_tas_ann_2deg.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='tas_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
filename='../outputs/netcdf/PMIP3_piControl_tas_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='tas_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_18

In [8]:
def ensemble_diffence(pmip_v,experiment_name,variable_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    A_dict=ensemble_members_dict(variable_name,experiment_name)
    B_dict=ensemble_members_dict(variable_name,'piControl')

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    #Set up the holding array
    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
 
    #loop over models
    n=0
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                modellist.append(gcm)
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_a=expt_a_file[variable_name]
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                expt_b=expt_b_file[variable_name]
                diff=expt_a-expt_b
                this_regridder=xe.Regridder(expt_a_file,grid_1x1,'bilinear', reuse_weights=True,periodic=True) 
                diff_1x1=this_regridder(diff) 
                expt_a_file.close()
                expt_b_file.close()
                data[n,:,:]=diff_1x1
                n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      

    return modellist

In [9]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_jja_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_jja'
model_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
model_list


Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM']

In [10]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_djf_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)
diff_list


Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


['AWI-ESM-1-1-LR',
 'CESM2',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM']

In [11]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_ann_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


In [12]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [13]:
filename='../outputs/netcdf/PMIP4_mh_diff_pr_ann_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)


Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


In [14]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_jja_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [34]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_jja_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


In [15]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_djf_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [33]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_djf_2deg.nc'
pmip_v='PMIP4'
experiment_name='midHolocene-cal-adj'
variable_name='tas_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_180x288_180x360_peri.nc
Reuse existing file: bilinear_80x180_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_120x180_180x360_peri.nc
Reuse existing file: bilinear_143x144_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc
Reuse existing file: bilinear_96x144_180x360_peri.nc


In [16]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_jja_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_jja'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [17]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_djf_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene-cal-adj'
variable_name='pr_spatialmean_djf'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [18]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [19]:
filename='../outputs/netcdf/PMIP3_mh_diff_pr_ann_2deg.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
diff_list=ensemble_diffence(pmip_v,experiment_name,variable_name,filename)

Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_192x288_180x360_peri.nc
Reuse existing file: bilinear_128x256_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_56x64_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc
Reuse existing file: bilinear_60x128_180x360_peri.nc
Reuse existing file: bilinear_108x128_180x360_peri.nc
Reuse existing file: bilinear_90x144_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_145x192_180x360_peri.nc
Reuse existing file: bilinear_96x96_180x360_peri.nc
Reuse existing file: bilinear_64x128_180x360_peri.nc
Reuse existing file: bilinear_96x192_180x360_peri.nc
Reuse existing file: bilinear_160x320_180x360_peri.nc


In [20]:
# figure 2 , averaged bands

In [21]:
modellist4= pd.read_csv('PMIP4_modellist.csv',skipinitialspace=True,header=0)['model']
modellist3= pd.read_csv('PMIP3_modellist.csv',skipinitialspace=True,header=0)['model']

In [22]:
#Weighted array
#equator=0: cos
#equatro=90: sin
#Weighted array
#equator=0: cos
#equatro=90: sin
w=[]
a_LAT=180
a_LON=360
for l in range(0,a_LAT): #lat No.
        lat=l*180/a_LAT
        lats=np.deg2rad(lat)
        we=np.sin(lats)
        w.append(we)

a2=np.ones((a_LAT,a_LON)) #lat X lon
for r in range(0,a_LAT):
    for c in range(0,a_LON):
        a2[r,c]=a2[r,c]*w[r]



In [23]:
#This function creates zonal means. I'm sure it could be written more succinctly
def zonal(filename,variable_name,pmip_v): 
    modellist=pd.read_csv('%s_modellist.csv'%pmip_v,skipinitialspace=True,header=0)['model']
    x=np.arange(0,181,30)
    this_file=xr.open_dataset(filename,decode_times=False)
    gcm_dict=dict(zip(this_file["modellist"].values,this_file["gcms"].values))
    data4={}
    for m in modellist:
        if m in this_file["modellist"].values:
            data4[gcm_dict[m]]=[]
            for i in range(len(x)-1):
                this_var=this_file[variable_name]
                men=this_var[gcm_dict[m]][x[i]:x[i+1]]
                xx=np.average(men,weights=a2[x[i]:x[i+1]])
                data4[gcm_dict[m]].append(xx)

    return data4


In [24]:
filename='../outputs/netcdf/PMIP4_mh_diff_tas_ann_2deg.nc'
data=zonal(filename,"tas_spatialmean_ann",'PMIP4')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP4_mh_diff_zonal_tas_ann.csv')

In [25]:
filename='../outputs/netcdf/PMIP4_piControl_tas_ann_2deg.nc'
data=zonal(filename,"tas_spatialmean_ann",'PMIP4')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP4_piControl_zonal_tas_ann.csv')

In [26]:
filename='../outputs/netcdf/PMIP3_mh_diff_tas_ann_2deg.nc'
data=zonal(filename,'tas_spatialmean_ann','PMIP3')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP3_mh_diff_zonal_tas_ann.csv')

In [27]:
filename='../outputs/netcdf/PMIP3_piControl_tas_ann_2deg.nc'
data=zonal(filename,'tas_spatialmean_ann','PMIP3')
pd.DataFrame(data).to_csv('../outputs/csv/PMIP3_piControl_zonal_tas_ann.csv')

In [28]:
# Monsoon files

In [29]:
def monsoon_cal(monsoon_name,experiment_name,pmip_v):
    monsoonlist=['NAF','EAS','AUSMC','NAMS','SAF','SAMS','SAS']
    data={}
    rainfall_name='monsoon_rain_%s' %monsoon_name
    area_name='monsoon_area_%s' %monsoon_name
    A_dict=ensemble_members_dict(rainfall_name,experiment_name)
    B_dict=ensemble_members_dict(rainfall_name,'piControl')
    for gcm in A_dict:
        if gcm in B_dict:
            if gcm in pmip[pmip_v]:
                expt_a_file=xr.open_dataset(A_dict.get(gcm),decode_times=False)
                expt_rain=expt_a_file[rainfall_name]
                expt_mean_rain=np.nanmean(expt_rain)
                expt_std_rain=np.nanstd(expt_rain)
                expt_area=expt_a_file[area_name]
                expt_mean_area=np.nanmean(expt_area)
                expt_std_area=np.nanstd(expt_area)
                expt_water=expt_rain*expt_area
                expt_mean_water=np.nanmean(expt_water)
                expt_b_file=xr.open_dataset(B_dict.get(gcm),decode_times=False)
                pi_rain=expt_b_file[rainfall_name]
                pi_mean_rain=np.nanmean(pi_rain)
                pi_std_rain=np.nanstd(pi_rain)
                pi_area=expt_b_file[area_name]
                pi_mean_area=np.nanmean(pi_area)
                pi_std_area=np.nanstd(pi_area)
                pi_water=pi_rain*pi_area
                pi_mean_water=np.nanmean(pi_water)
                pav=(expt_mean_rain-pi_mean_rain)*100/pi_mean_rain
                psd=(expt_std_rain-pi_std_rain)*100/pi_std_rain
                aav=(expt_mean_area-pi_mean_area)*100/pi_mean_area
                asd=(expt_std_area-pi_std_area)*100/pi_std_area
                water=(expt_mean_water-pi_mean_water)*100/pi_mean_water
                data[gcm]=[pav,psd,aav,asd,water]
    f3='../outputs/csv/%s_%s.csv' %(pmip_v,monsoon_name)
    DD=pd.DataFrame(data)
    DD.to_csv(f3)
    return data

def monsoonchange(pmip_v,experiment_name):
    monsoonlist=['NAF','EAS','AUSMC','NAMS','SAF','SAMS','SAS',]
    modellistname='%s_modellist.csv'%pmip_v
    model_list4= pd.read_csv(modellistname,skipinitialspace=True,header=0)['model']
    pav_data4={}
    psd_data4={}
    aav_data4={}
    asd_data4={}
    water_data4={}
    for m in model_list4:
        pav_data4[m]=[]
        psd_data4[m]=[]
        aav_data4[m]=[]
        asd_data4[m]=[]
        water_data4[m]=[]
    for monsoon_name in monsoonlist:
        data4=monsoon_cal(monsoon_name,experiment_name,pmip_v)
        for m in model_list4:
            if m in data4:
                pav_data4[m].append(data4[m][0])
                psd_data4[m].append(data4[m][1])
                aav_data4[m].append(data4[m][2])
                asd_data4[m].append(data4[m][3])
                water_data4[m].append(data4[m][4])
            else:
                pav_data4[m].append(np.nan)
                psd_data4[m].append(np.nan)
                aav_data4[m].append(np.nan)
                asd_data4[m].append(np.nan)
                water_data4[m].append(np.nan)
                
    pd.DataFrame(pav_data4).to_csv('../outputs/csv/%s_pav.csv'%pmip_v)
    pd.DataFrame(aav_data4).to_csv('../outputs/csv/%s_aav.csv'%pmip_v)
    pd.DataFrame(psd_data4).to_csv('../outputs/csv/%s_psd.csv'%pmip_v)
    pd.DataFrame(asd_data4).to_csv('../outputs/csv/%s_asd.csv'%pmip_v)
    pd.DataFrame(water_data4).to_csv('../outputs/csv/%s_totwater.csv'%pmip_v)




In [30]:
monsoonchange('PMIP4','midHolocene')
monsoonchange('PMIP3','midHolocene')

In [31]:
def calaverage(pmip_v):
    varlist=['pav','psd','aav','asd','totwater']
    for v in varlist:    
        mon=v
        data=pd.read_csv('../outputs/csv/%s_%s.csv'%(pmip_v,mon),skipinitialspace=True,header=0)
        D=np.array(data)[:,1:]
        ave=np.average(D,axis=1)
        data['average']=ave
        data.to_csv('../outputs/csv/%s_%s.csv'%(pmip_v,mon))

In [32]:
calaverage('PMIP4')
calaverage('PMIP3')